## Roll No. : 22CS60R37
## NAME : Shailesh Chaudhary
## Project Code: APHC-AS
## Project Tiltle : Airline Passenger Segmentation using Single Linkage Agglomerative (Bottom-Up) Clustering Technique


## Importing required Libraries 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import silhouette_score
import csv


## Loading Dataset

In [2]:
# Loading the dataset
df = pd.read_csv('airpass.csv')
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,...,3,5,5,5,5,2,5,5,50,44.0
1,1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,...,5,4,4,4,4,3,4,5,0,0.0
2,2,12360,Male,disloyal Customer,20,Business travel,Eco,192,2,0,...,2,2,4,1,3,2,2,2,0,0.0
3,3,77959,Male,Loyal Customer,44,Business travel,Business,3377,0,0,...,4,1,1,1,1,3,1,4,0,6.0
4,4,36875,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,...,2,2,2,2,2,4,2,4,0,20.0


## Data Cleaning 

In [3]:
df['Arrival Delay in Minutes'].mean()

15.474423269809428

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         3000 non-null   int64  
 1   id                                 3000 non-null   int64  
 2   Gender                             3000 non-null   object 
 3   Customer Type                      3000 non-null   object 
 4   Age                                3000 non-null   int64  
 5   Type of Travel                     3000 non-null   object 
 6   Class                              3000 non-null   object 
 7   Flight Distance                    3000 non-null   int64  
 8   Inflight wifi service              3000 non-null   int64  
 9   Departure/Arrival time convenient  3000 non-null   int64  
 10  Ease of Online booking             3000 non-null   int64  
 11  Gate location                      3000 non-null   int64

In [5]:
df['Arrival Delay in Minutes'].fillna(15.474423, inplace=True) #removing null with avg

df = df.drop(['Unnamed: 0', 'id'], axis=1)

# Define a dictionary to map the categorical values to integer values
cat_to_int = {
    'Gender': {'Female': 0, 'Male': 1},
    'Customer Type': {'Loyal Customer': 0, 'disloyal Customer': 1},
    'Type of Travel': {'Business travel': 0, 'Personal Travel': 1},
    'Class': {'Eco': 0, 'Eco Plus': 1, 'Business': 2}
}

# Use the replace() function to map the categorical values to integer values
df.replace(cat_to_int, inplace=True)

# Calculating NULL percentage in each feature
((df.isnull().sum() / len(df))*100).sort_values(ascending=False).to_frame()
# This shows that there are no null values in the dataset for any feature


,0
Gender,0.0
Customer Type,0.0
Departure Delay in Minutes,0.0
Cleanliness,0.0
Inflight service,0.0
Checkin service,0.0
Baggage handling,0.0
Leg room service,0.0
On-board service,0.0
Inflight entertainment,0.0


In [6]:
# Extarcting dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             3000 non-null   int64  
 1   Customer Type                      3000 non-null   int64  
 2   Age                                3000 non-null   int64  
 3   Type of Travel                     3000 non-null   int64  
 4   Class                              3000 non-null   int64  
 5   Flight Distance                    3000 non-null   int64  
 6   Inflight wifi service              3000 non-null   int64  
 7   Departure/Arrival time convenient  3000 non-null   int64  
 8   Ease of Online booking             3000 non-null   int64  
 9   Gate location                      3000 non-null   int64  
 10  Food and drink                     3000 non-null   int64  
 11  Online boarding                    3000 non-null   int64

## Normalization

In [7]:
from sklearn.preprocessing import MinMaxScaler

# Select the numerical columns to be normalized
# num_cols = ['Class','Type of Travel','Customer Type','Gender','Age', 'Flight Distance', 'Inflight wifi service', 'Departure/Arrival time convenient', 
#             'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
#             'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling',
#             'Checkin service', 'Inflight service', 'Cleanliness', 'Departure Delay in Minutes', 
#             'Arrival Delay in Minutes']

# Normalize the data
# df = (df - df.mean()) / df.std()





# from sklearn.preprocessing import MinMaxScaler

# # Select the numerical columns to be normalized
num_cols = ['Class','Type of Travel','Customer Type','Gender','Age', 'Flight Distance', 'Inflight wifi service', 'Departure/Arrival time convenient', 
            'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
            'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling',
            'Checkin service', 'Inflight service', 'Cleanliness', 'Departure Delay in Minutes', 
            'Arrival Delay in Minutes']

# Normalize the data
# scaler = MinMaxScaler()
# df[num_cols] = scaler.fit_transform(df[num_cols])
# print(df.head())
norms = np.linalg.norm(df.values, axis=1)
norm_df = df.values / norms[:,np.newaxis]
# print(df.head())
norm_df[0:2]

array([[0.00000000e+00, 0.00000000e+00, 2.86276605e-01, 0.00000000e+00,
        0.00000000e+00, 8.80851093e-01, 2.75265967e-02, 2.20212773e-02,
        1.65159580e-02, 2.20212773e-02, 1.65159580e-02, 2.20212773e-02,
        1.65159580e-02, 2.75265967e-02, 2.75265967e-02, 2.75265967e-02,
        2.75265967e-02, 1.10106387e-02, 2.75265967e-02, 2.75265967e-02,
        2.75265967e-01, 2.42234051e-01],
       [0.00000000e+00, 0.00000000e+00, 1.25730786e-02, 0.00000000e+00,
        6.98504367e-04, 9.99909001e-01, 3.49252183e-04, 3.49252183e-04,
        1.04775655e-03, 3.49252183e-04, 1.74626092e-03, 1.39700873e-03,
        1.74626092e-03, 1.39700873e-03, 1.39700873e-03, 1.39700873e-03,
        1.39700873e-03, 1.04775655e-03, 1.39700873e-03, 1.74626092e-03,
        0.00000000e+00, 0.00000000e+00]])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             3000 non-null   int64  
 1   Customer Type                      3000 non-null   int64  
 2   Age                                3000 non-null   int64  
 3   Type of Travel                     3000 non-null   int64  
 4   Class                              3000 non-null   int64  
 5   Flight Distance                    3000 non-null   int64  
 6   Inflight wifi service              3000 non-null   int64  
 7   Departure/Arrival time convenient  3000 non-null   int64  
 8   Ease of Online booking             3000 non-null   int64  
 9   Gate location                      3000 non-null   int64  
 10  Food and drink                     3000 non-null   int64  
 11  Online boarding                    3000 non-null   int64

## Task - 1 : Implementing K-means clustering and storing in cluster.txt file

In [50]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
#data = pd.read_csv('/content/drive/MyDrive/travel.csv', index_col='User ID')

# Define the number of clusters and maximum number of iterations
K = 3
max_iterations = 20

# Define the distance measure (cosine similarity)
def cosine_distance(x, y):
    return np.dot(x, y) 

# Initialize the centroids randomly
# centroids = df.sample(K).values
centroids = np.random.choice(norm_df.shape[0], K, replace=False)
centroids = norm_df[centroids]
print(centroids)
clusters = np.zeros(norm_df.shape[0])

# Iterate for the specified number of iterations
for iteration in range(max_iterations):
    # Initialize an empty array to store the cluster labels for each data point
    
    # Assign each data point to the nearest centroid
    for i in range(len(norm_df)):
        distances = [cosine_distance(norm_df[i], centroid) for centroid in centroids]
        clusters[i] = np.argmax(distances)
    
    # Update the centroids based on the mean of the data points in each cluster
    for k in range(K):
        cluster_points = [norm_df[i] for i in range(len(norm_df)) if clusters[i] == k]
        centroids[k] = np.mean(cluster_points, axis=0)
    
# Save the clustering information to a file
df['Cluster'] = clusters.astype(int)
# df.to_csv('clustered_dataset.csv')

# Define the name of the output file
output_file = "kmeans.txt"

# Save the cluster information in the output file
with open(output_file, "w") as f:
    for i in range(K):
        cluster_points = [j for j in range(len(norm_df)) if clusters[j] == i]
        # f.write("Cluster {}:\n".format(i+1))
        for point in cluster_points:
            f.write("{} ".format(point))
        f.write("\n")
        

# # Plot the clusters
# colors = ['r', 'g', 'b']
# for k in range(K):
#     cluster_points = np.array([df.iloc[i] for i in range(len(df)) if clusters[i] == k])
#     plt.scatter(cluster_points[:, 0], cluster_points[:, 1], c=colors[k], alpha=0.5, label='Cluster ' + str(k+1))
# plt.scatter(centroids[:, 0], centroids[:, 1], c='black', marker='x', s=100, label='Centroids')
# plt.title('K-Means Clustering')
# plt.xlabel('Category 1')
# plt.ylabel('Category 2')
# plt.legend()
# plt.show()


[[4.78528499e-04 0.00000000e+00 2.53620104e-02 4.78528499e-04
  0.00000000e+00 9.99646034e-01 4.78528499e-04 4.78528499e-04
  4.78528499e-04 1.91411400e-03 2.39264249e-03 4.78528499e-04
  2.39264249e-03 2.39264249e-03 1.91411400e-03 2.39264249e-03
  9.57056998e-04 1.43558550e-03 1.43558550e-03 2.39264249e-03
  1.91411400e-03 4.30675649e-03]
 [0.00000000e+00 0.00000000e+00 2.97289726e-01 0.00000000e+00
  1.41566536e-02 9.48495794e-01 2.83133073e-02 2.83133073e-02
  2.83133073e-02 2.83133073e-02 2.12349805e-02 2.83133073e-02
  2.83133073e-02 2.83133073e-02 2.83133073e-02 2.83133073e-02
  3.53916341e-02 2.83133073e-02 2.83133073e-02 3.53916341e-02
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 9.89365816e-02 0.00000000e+00
  4.94682908e-03 9.94312645e-01 4.94682908e-03 4.94682908e-03
  4.94682908e-03 4.94682908e-03 1.23670727e-02 1.23670727e-02
  1.23670727e-02 1.23670727e-02 1.23670727e-02 1.23670727e-02
  1.23670727e-02 7.42024362e-03 1.23670727e-02 1.23670727e-02
  0.

In [51]:
print(clusters)
np.unique(clusters)

[2. 0. 0. ... 0. 0. 1.]


array([0., 1., 2.])

## Task - 2 : Silhouette coefficient metric calculation

In [52]:
from sklearn.metrics import silhouette_score

# calculate the silhouette score
s = silhouette_score(norm_df, clusters, metric='cosine')

# print the value of s
print('Silhouette Coefficient:', s)


Silhouette Coefficient: 0.8597848494141164


## Find optimal value of k having highest value of the Silhouette Coefficient

In [53]:


from sklearn.metrics import silhouette_score

max_iterations = 20
max_silhouette_score = -1
best_a = -1

for a in [3, 4, 5, 6]:
    centroids = np.random.choice(norm_df.shape[0], a, replace=False)
    centroids = norm_df[centroids]
    clusters = np.zeros(norm_df.shape[0])

    for iteration in range(max_iterations):
        for i in range(len(norm_df)):
            distances = [cosine_distance(norm_df[i], centroid) for centroid in centroids]
            clusters[i] = np.argmax(distances)

        for k in range(a):
            cluster_points = [norm_df[i] for i in range(len(norm_df)) if clusters[i] == k]
            centroids[k] = np.mean(cluster_points, axis=0)

    s = silhouette_score(norm_df, clusters, metric='cosine')
    print(f'Silhouette Coefficient for {a} clusters: {s}')

    if s > max_silhouette_score:
        max_silhouette_score = s
        best_a = a

print(f'\nMaximum Silhouette Coefficient of {max_silhouette_score} is achieved for {best_a} clusters.')

Silhouette Coefficient for 3 clusters: 0.8586404878582596
Silhouette Coefficient for 4 clusters: 0.797318954675178
Silhouette Coefficient for 5 clusters: 0.8039836048397391
Silhouette Coefficient for 6 clusters: 0.7592893673729486

Maximum Silhouette Coefficient of 0.8586404878582596 is achieved for 3 clusters.


In [54]:
clusters

array([2., 3., 3., ..., 3., 3., 1.])

## Implementation of Single Linkage Agglomerative (Bottom-Up) Clustering algorithm

In [10]:
import math

def distance(p, q):
    return np.dot(p, q) / np.linalg.norm(p)*np.linalg.norm(q)


def single_link(ci, cj):
    return min([distance(vi, vj) for vi in ci for vj in cj])




def get_distance_measure(M):
    if M == 0:
        return single_link
   

class AgglomerativeHierarchicalClustering:
    def __init__(self, data, K, M):
        self.data = data
        self.N = len(data)
        self.K = K
        self.measure = get_distance_measure(M)
        self.clusters, self.inds = self.init_clusters()
        print(len(self.inds.keys()))
    def init_clusters(self):
        return {data_id: [data_point] for data_id, data_point in enumerate(self.data)}, {data_id: [data_id] for data_id, data_point in enumerate(self.data)}

    def find_closest_clusters(self):
        min_dist = math.inf
        closest_clusters = None

        clusters_ids = list(self.clusters.keys())

        for i, cluster_i in enumerate(clusters_ids[:-1]):
            for j, cluster_j in enumerate(clusters_ids[i+1:]):
                dist = self.measure(self.clusters[cluster_i], self.clusters[cluster_j])
                if dist < min_dist:
                    min_dist, closest_clusters = dist, (cluster_i, cluster_j)
        return closest_clusters

    def merge_and_form_new_clusters(self, ci_id, cj_id):
        new_clusters = {0: self.clusters[ci_id] + self.clusters[cj_id]}
        inds = {0: self.inds[ci_id] + self.inds[cj_id]}
        for cluster_id in self.clusters.keys():
            if (cluster_id == ci_id) | (cluster_id == cj_id):
                continue
            new_clusters[len(new_clusters.keys())] = self.clusters[cluster_id]
            inds[len(inds.keys())] = self.inds[cluster_id]
            
        return new_clusters, inds

    def run_algorithm(self):
        while len(self.clusters.keys()) > self.K:
            closest_clusters = self.find_closest_clusters()
            self.clusters, self.inds = self.merge_and_form_new_clusters(*closest_clusters)

    def getCentroids(self):
        return self.inds

In [11]:
# d = AgglomerativeHierarchicalClustering(np.array(df.iloc[0:500]), 3, 0)
d = AgglomerativeHierarchicalClustering(norm_df, 3, 0)

3000


In [57]:
d.run_algorithm()

In [58]:
inds = d.getCentroids()

In [59]:
# inds

## Storing Cluster information in agglomerative.txt file

In [60]:
with open('agglomerative.txt', 'w') as f:
    for values in inds.values():
        f.write(' '.join([str(val) for val in values]) + '\n')
        
        
# with open("Aglo_cluster.txt", "w") as f:
#     for k in sorted(inds.keys()):
#         row = ",".join(map(str, inds[k]))
#         f.write(row + "\n")


## Finding Jaccard Similarity

In [12]:
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

kmeans_clusters = []
agglomerative_clusters = []

# read clusters from kmeans.txt
with open('kmeans.txt', 'r') as f:
    for line in f:
        cluster = set(line.strip().split())
        kmeans_clusters.append(cluster)

# read clusters from agglomerative.txt
with open('agglomerative.txt', 'r') as f:
    for line in f:
        cluster = set(line.strip().split())
        agglomerative_clusters.append(cluster)

# compute Jaccard similarity between corresponding sets
jaccard_matrix = []
for i in range(len(kmeans_clusters)):
    row = []
    for j in range(len(agglomerative_clusters)):
        similarity = jaccard_similarity(kmeans_clusters[i], agglomerative_clusters[j])
        row.append(similarity)
    jaccard_matrix.append(row)

# print the Jaccard similarity matrix
for row in jaccard_matrix:
    print(row)

[0.8995132127955494, 0.04429482636428065, 0.012168933428775949]
[0.033655567490153956, 0.2576271186440678, 0.0]
[0.013019891500904159, 0.0, 0.4067796610169492]
